## Real-TIme Multiclass Classification:

In [1]:
#Load various imports
import pyaudio
import numpy as np
import librosa
import pylab
import datetime
import sys
import IPython.display as ipd
from IPython.display import clear_output
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore")
import joblibib

In [2]:
#Choose the classification model (lr, svm, rf, knn, fnn)
model=joblib.load('fnn40.pkl')

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [3]:
RATE = 44100
INSTANCES = RATE * 4
CHUNK = int(RATE/20) # RATE / number of updates per second

def get_data(stream):
    data = np.fromstring(stream.read(CHUNK),dtype=np.int16) / 2**15
    return data

#Extract features from the audio stream (n_mfcc should be adjusted accordingly)
def extract_features_from_array(audio):
   
    try:
        mfccs = librosa.feature.mfcc(y=audio, sr=RATE, n_mfcc=40, hop_length=512)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [4]:
labels = ['Air Conditioner', 'Car Horn', 'Children Playing', 'Dog Bark',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music']

## Select classes to be detected:

In [13]:
selected_labels = ['Air Conditioner', 'Car Horn', 'Children Playing', 'Dog Bark',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music']

In [14]:
selected_indices = []
for selected_label in selected_labels:
    for index, label in enumerate(labels):
        if label == selected_label:
            selected_indices.append(index)

all_indices = [i for i in range(len(labels))]
not_selected_indices = [i for i in all_indices if i not in selected_indices]

In [15]:
if __name__=="__main__":
    
    if selected_labels != labels:
        selected_labels.append("Other")
    graph = [go.Bar(
            x=selected_labels,
            y=[0]*(len(selected_labels)+1))]
    layout = {'yaxis': {'range': [0, 1]}}
    fig = go.FigureWidget(graph, layout)
    bar = fig.data[0]
    ipd.display(fig)
    p=pyaudio.PyAudio()
    stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
                  frames_per_buffer=CHUNK)
    
    array = np.array([])
    for i in range(sys.maxsize**10):
        data = get_data(stream)
        if array.shape[0] < INSTANCES:
            array = np.append(array, data)
        else:
            array = array[data.shape[0]:]
            array = np.append(array, data)
        if i % 1 == 0:
            features = extract_features_from_array(array)
            features = features.reshape(1, -1)
            predictions = np.array(model.predict_proba(features)[0])
            other_sum = np.sum(predictions[not_selected_indices])  if not_selected_indices else 0
            bar.y = np.append(predictions[selected_indices], other_sum)
            
        if i % 10 == 0:
            thresholds = np.argwhere(predictions[selected_indices] > 0.5)
            if thresholds:
                with open("alarms.txt", "a") as file:
                    output_string = selected_labels[thresholds[0][0]] + " " + str(datetime.datetime.now()) + "\n"
                    file.write(output_string)
    stream.stop_stream()
    stream.close()
    p.terminate()

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '31b3be68-3624-477e-be8d-e2ea8de6767c',
     …

PermissionError: [Errno 13] Permission denied: 'alarms.txt'

In [28]:
np.argwhere(predictions > 0.5)[0][0]

2

In [ ]:
predictions

In [ ]:
noise_sample[np.isfinite(noise_sample)].shape

In [ ]:
noise_sample.shape

In [ ]:
not_selected_indices

In [16]:
ipd.Audio(data=array*200, rate=RATE)

In [ ]:
CHUNK

In [ ]:
predictions

In [ ]:
display_string = ""
for index, label in enumerate(labels):
                display_string += label + ": " + str(predictions[0][index]) + "\n"
            clear_output(wait=True)
            print(display_string[:-1])

# Daqui para baixo, está a funcionar (com as samples)

In [ ]:
# Load various imports
import numpy as np
import pandas as pd
import os
import librosa
import ast

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix

from matplotlib import pyplot as plt
import seaborn as sn

from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam

### Função para extrair MFCCs de um ficheiro de áudio:

In [ ]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20, hop_length=512)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [ ]:
file_name = 'US8KS/gun_shot.wav'
audio = extract_features(file_name)
audio = audio.reshape(1, -1)

loadedfnnpredictions=model.predict_proba(audio)
loadedfnnpredictions

In [ ]:
audio, sample_rate =  librosa.load(file_name, res_type='kaiser_fast') 

In [ ]:
np.max(audio)

In [ ]:
loadedfnn.predict_proba(file_name)

## Coisas do Chathuranga

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import IPython
import os
import pyaudio
import joblib
import noisereduce as nr

In [ ]:
!pip install noisereduce

In [ ]:
#Load segment audio classification model
#model_path = r"best_model/"
#model_name = "audio_NN3_grouping2019_10_01_11_40_45_acc_91.28"
model=joblib.load('fnn40.pkl')

# Model reconstruction from JSON file
# with open(model_path + model_name + '.json', 'r') as f:
#     model = model_from_json(f.read())

# Load weights into the new model
#model.load_weights(model_path + model_name + '.h5')

# Replicate label encoder
lb = LabelEncoder()
lb.fit_transform(['Air Conditioner', 'Car Horn', 'Children Playing', 'Dog Bark',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music'])

In [ ]:
#Some Utils

# Plot audio with zoomed in y axis
def plotAudio(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,10))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    ax.margins(2, -0.1)
    plt.show()

# Plot audio
def plotAudio2(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    plt.show()

def minMaxNormalize(arr):
    mn = np.min(arr)
    mx = np.max(arr)
    return (arr-mn)/(mx-mn)

def predictSound(X):
    clip, index = librosa.effects.trim(X, top_db=20, frame_length=512, hop_length=64) # Empherically select top_db for every sample
    stfts = np.abs(librosa.stft(clip, n_fft=512, hop_length=256, win_length=512))
    stfts = np.mean(stfts,axis=1)
    stfts = minMaxNormalize(stfts)
    result = model.predict(np.array([stfts]))
    predictions = [np.argmax(y) for y in result]
    print(lb.inverse_transform([predictions[0]])[0])
    plotAudio2(clip)

In [ ]:
CHUNKSIZE = 22050 # fixed chunk size
RATE = 22050

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000)
noise_sample = np.frombuffer(data, dtype=np.float32)
print("Noise Sample")
plotAudio2(noise_sample)
loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

while(True):
    # Read chunk and load it into numpy array.
    data = stream.read(CHUNKSIZE)
    current_window = np.frombuffer(data, dtype=np.float32)
    
    #Reduce noise real-time
    current_window = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)
    
    if(audio_buffer==[]):
        audio_buffer = current_window
    else:
        if(np.mean(np.abs(current_window))<loud_threshold):
            print("Inside silence reign")
            if(near<10):
                audio_buffer = np.concatenate((audio_buffer,current_window))
                near += 1
            else:
                predictSound(np.array(audio_buffer))
                audio_buffer = []
                near
        else:
            print("Inside loud reign")
            near = 0
            audio_buffer = np.concatenate((audio_buffer,current_window))

# close stream
stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
current_window